In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [3]:
Deaths_df=pd.read_csv(r"C:\Users\NehaS\Desktop\CS418\MEASURESOFBIRTHANDDEATH.csv")
#Deaths_IL=Deaths_df[Deaths_df['CHSI_State_Name']=='Illinois']
#Deaths_IL

In [4]:
Death_Infant=Deaths_df[['CHSI_County_Name','LBW','VLBW','Premature','Under_18','Over_40','Unmarried','Late_Care','Infant_Mortality']]
Death_Infant=Death_Infant.replace([-2222.2,-1111.1],np.NaN)


### K-Means Clustering and Multinomial Bayes Classifier for Infant Mortality Rate

In [27]:
Death_Infant=Death_Infant[Death_Infant['Infant_Mortality'].notnull()]
Death_Infant=Death_Infant.dropna()
X=np.array(Death_Infant[['Infant_Mortality','LBW','VLBW','Premature','Over_40','Unmarried','Late_Care']])
model = KMeans(n_clusters=2, random_state=1)
model.fit(X)
pred = model.predict(X)
levels=['2','1']
pred_val=[levels[x] for x in pred]
print("The 3 means for calculating low, medium and high healthcare areas using clustering are:")
cluster_cent=zip(levels,model.cluster_centers_)
print([x for x in cluster_cent])
Death_Infant['Death_Level']=pred_val
Death_Infant


The 3 means for calculating low, medium and high healthcare areas using clustering are:
[('2', array([ 9.21536585,  9.08670732,  1.73926829, 13.86182927,  1.43878049,
       46.1647561 , 22.39170732])), ('1', array([ 6.66405767,  7.10020597,  1.22178167, 11.30720906,  1.7861483 ,
       27.42296601, 15.55545829]))]


,CHSI_County_Name,LBW,VLBW,Premature,Under_18,Over_40,Unmarried,Late_Care,Infant_Mortality,Death_Level
0,Autauga,8.1,1.6,14.3,5.1,1.4,28.0,12.9,5.9,1
1,Baldwin,8.6,1.9,15.3,4.3,1.9,25.4,15.3,7.1,1
2,Barbour,11.0,1.9,16.7,7.7,1.1,50.2,26.0,7.1,2
3,Bibb,8.7,1.7,13.7,6.5,0.9,30.2,16.3,9.8,1
4,Blount,7.6,1.5,12.8,5.0,1.3,19.3,16.7,9.0,1
...,...,...,...,...,...,...,...,...,...,...
3136,Sweetwater,8.8,1.3,11.4,3.8,1.3,31.9,23.3,10.2,1
3137,Teton,8.0,0.8,11.0,2.5,4.5,21.4,10.9,3.5,1
3138,Uinta,11.4,1.1,14.4,5.2,1.3,25.5,13.8,7.0,1
3139,Washakie,7.3,0.4,10.8,4.1,1.2,23.2,14.9,9.1,1


In [28]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X1=Death_Infant[['LBW','VLBW','Premature','Under_18','Over_40','Unmarried','Late_Care']].to_numpy()
y=Death_Infant['Death_Level'].to_numpy()
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.10,random_state=5)
clf = MultinomialNB()
clf.fit(X1_train,y_train)
inf_pred=clf.predict(X1_test)
print(accuracy_score(y_test,inf_pred))

0.8592057761732852


### Backward Elimination to improve model accuracy

In [29]:
import statsmodels.api as sm
X_opt =X1[:,[0,1,2,3,4,5,6]]
OLS_res = sm.OLS(endog=Death_Infant['Infant_Mortality'], exog=X_opt).fit()
print(OLS_res.summary())

                                 OLS Regression Results                                
Dep. Variable:       Infant_Mortality   R-squared (uncentered):                   0.918
Model:                            OLS   Adj. R-squared (uncentered):              0.918
Method:                 Least Squares   F-statistic:                              4431.
Date:                Wed, 22 Apr 2020   Prob (F-statistic):                        0.00
Time:                        14:39:55   Log-Likelihood:                         -6188.4
No. Observations:                2762   AIC:                                  1.239e+04
Df Residuals:                    2755   BIC:                                  1.243e+04
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [30]:
import statsmodels.api as sm
X_opt =X1[:,[0,1,2,4,5,6]]
OLS_res = sm.OLS(endog=Death_Infant['Infant_Mortality'], exog=X_opt).fit()
print(OLS_res.summary())

                                 OLS Regression Results                                
Dep. Variable:       Infant_Mortality   R-squared (uncentered):                   0.918
Model:                            OLS   Adj. R-squared (uncentered):              0.918
Method:                 Least Squares   F-statistic:                              5171.
Date:                Wed, 22 Apr 2020   Prob (F-statistic):                        0.00
Time:                        14:39:57   Log-Likelihood:                         -6188.6
No. Observations:                2762   AIC:                                  1.239e+04
Df Residuals:                    2756   BIC:                                  1.242e+04
Df Model:                           6                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

### Accuracy score after performing Backward Elimination

In [31]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X2=Death_Infant[['LBW','VLBW','Premature','Over_40','Unmarried','Late_Care']].to_numpy()
y2=Death_Infant['Death_Level'].to_numpy()
X2_train, X2_test, y2_train, y2_test = train_test_split(X1, y, test_size=0.20,random_state=5)
clf = MultinomialNB()
clf.fit(X2_train,y2_train)
inf_pred2=clf.predict(X2_test)
print(accuracy_score(y2_test,inf_pred2))

0.8625678119349005
